In [8]:
# 반드시 이 셀을 실행하여 Kaggle 데이터 소스를 가져오세요.
# 이후 이 셀은 삭제해도 됩니다.
# 참고: 이 노트북 환경은 Kaggle의 파이썬 환경과 다르므로 일부 라이브러리가 없을 수 있습니다.
import kagglehub
aadityabansalcodes_telecommunications_industry_customer_churn_dataset_path = kagglehub.dataset_download('aadityabansalcodes/telecommunications-industry-customer-churn-dataset')

print('데이터 소스 가져오기 완료.')

데이터 소스 가져오기 완료.


<body>
    <h1>데이터셋 소개</h1>
<p>
이 텔레콤 이탈 데이터셋은 한 통신사 고객의 정보와 이탈(서비스 해지) 여부를 담고 있습니다. 고객의 인구통계(나이, 성별 등)와 서비스 이용 데이터(통화 횟수, 분수, 결제 방식 등) 등 다양한 특성이 포함되어 있습니다.
</p>

<p>이 데이터셋은 7043개의 샘플과 21개의 특성으로 구성되어 있으며, 고객 이탈 예측을 위한 머신러닝 및 데이터 분석의 벤치마크로 자주 사용됩니다. 이 데이터를 활용해 이탈 위험 고객을 식별하고, 이탈 방지 전략을 세워 고객 유지율과 매출을 높일 수 있습니다.</p>

<p>자세한 내용은 아래 링크를 참고하세요:</p>
<ul>
<li><a href="https://www.kaggle.com/blastchar/telco-customer-churn">Kaggle</a></li>
<li><a href="https://archive.ics.uci.edu/ml/datasets/Telecom+Customer+Churn">UCL</a></li>
</ul>
</body>
</html>

**특성 설명**

---
- **customerID**: 고객 ID
- **gender**: 고객 성별(남/여)
- **Senior Citizen**: 시니어(고령자) 여부 (1, 0)
- **Partner**: 배우자(파트너) 여부 (Yes, No)
- **Dependents**: 부양가족 여부 (Yes, No)
- **tenure**: 회사와 함께한 개월 수
- **Phone Service**: 전화 서비스 가입 여부 (Yes, No)
- **Multiple Lines**: 복수 회선 여부 (Yes, No, No phone service)
- **Internet Service**: 인터넷 서비스 제공자 (DSL, Fiber optic, No)
- **Online Security**: 온라인 보안 서비스 여부 (Yes, No, No internet service)
- **Online Backup**: 온라인 백업 서비스 여부 (Yes, No, No internet service)
- **Device Protection**: 기기 보호 서비스 여부 (Yes, No, No internet service)
- **Tech Support**: 기술 지원 서비스 여부 (Yes, No, No internet service)
- **StreamingTV**: TV 스트리밍 서비스 여부 (Yes, No, No internet service)
- **Streaming**: 영화 스트리밍 서비스 여부 (Yes, No, No internet service)
- **Contract**: 계약 유형 (월별, 1년, 2년)
- **Paperless Billing**: 무지(종이 없는) 청구서 여부 (Yes, No)
- **Payment Method**: 결제 방식 (전자수표, 우편수표, 자동이체, 신용카드 자동이체)
- **Monthly Charges**: 월별 청구 금액
- **Total Charges**: 총 청구 금액
- **Churn Label**: 이탈 여부 (Yes 또는 No)

## 데이터 탐색을 위한 초기 계획

**1. 데이터 개요:**
- 데이터셋을 불러오고 구조를 확인합니다.
- 주요 컬럼(고객 인구통계, 이탈 여부 등) 파악

**2. 기술 통계 분석:**
- 수치형 컬럼의 요약 통계 확인
- 이탈 분포 시각화 및 트렌드 파악

**3. 범주형 분석:**
- 범주형 변수 탐색
- 이탈률에 미치는 영향 분석

**4. 이탈 사유 분석:**
- 이탈 사유 조사
- 주요 이탈 원인 파악

**5. 상관관계 분석**

**6. 가설 검정**
- 가설(1): 전화 서비스가 이탈에 영향이 있는가?
- 가설(2): 계약 유형이 이탈에 영향이 있는가?
- 가설(3): 고령자 여부가 이탈에 영향이 있는가?

**8. 추가 분석 방향 제안**

**8. 데이터 품질 요약**

In [10]:
# 데이터프레임으로 데이터 읽기
import pandas as pd
df = pd.read_csv(aadityabansalcodes_telecommunications_industry_customer_churn_dataset_path + '/Telco_customer_churn.csv')
df.head(4)

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
n_rows, n_columns = df.shape
print(f"컬럼 수: {n_columns}개\n행 수: {n_rows}개")

# 기술 통계 분석

<font color=#0b5bdb>**1. Total charge는 float이어야 하지만 object로 표시되고 있습니다. float으로 변환하겠습니다.**</font>

In [ ]:
total_charge = df["Total Charges"]
missing = total_charge[~total_charge.str.replace(".", "").str.isdigit()]
print("총 청구금액 결측치 개수: ", len(missing))
missing.head()

---
<p>위 데이터프레임에서 볼 수 있듯이, 총 청구금액 컬럼에 일부 결측값이 존재합니다.</p>

In [ ]:
# 총 청구금액 컬럼을 숫자형으로 변환
df["Total Charges"] = df["Total Charges"].apply(pd.to_numeric, errors="coerce")

<font color=#0b5bdb>**2. 수치형 컬럼의 요약 통계 출력**</font>

위 표에서 총 청구금액이 범주형으로 표시되고 있습니다. 이는 잘못된 것으로, 숫자형 컬럼이어야 합니다. 추후 처리하겠습니다.

<font color=#0b5bdb>**3. 수치형 컬럼과 이탈 여부의 바차트 분석**</font>

**결과 해석**

---
#### 가입 개월 수:
- 이탈하지 않은 고객(`Churn Label` = "No")의 평균 가입 개월 수는 약 37.57개월입니다.
- 이탈한 고객(`Churn Label` = "Yes")의 평균 가입 개월 수는 약 17.98개월로 더 낮습니다.
- 즉, 오래 가입한 고객일수록 이탈 가능성이 낮음을 시사합니다.

#### 월별 청구금액:
- 이탈하지 않은 고객의 평균 월 청구금액은 약 `$61.27`입니다.
- 이탈한 고객의 평균 월 청구금액은 약 `$74.44`로 약간 더 높습니다.
- 이탈 고객이 평균적으로 더 높은 월 요금을 내는 경향이 있습니다.

#### 총 청구금액:
- 이탈하지 않은 고객의 평균 총 청구금액은 약 `$2,554.77`입니다.
- 이탈한 고객의 평균 총 청구금액은 약 `$1,531.80`로 더 낮습니다.
- 총 청구금액이 높을수록 이탈 가능성이 낮음을 시사합니다.

#### CLTV(고객 생애 가치):
- 이탈하지 않은 고객의 평균 CLTV는 약 `$4,490.92`입니다.
- 이탈한 고객의 평균 CLTV는 약 `$4,149.41`로 약간 더 낮습니다.
- 즉, 장기 고객이 더 높은 CLTV를 보입니다.

# 범주형 분석

<font color=#0b5bdb>**1. 데이터셋 내 고유 범주 확인**</font>

<font color=#0b5bdb>**2. 이탈 여부별 범주 빈도 분석**</font>

<font color=#0b5bdb>**3. 범주별 이탈 빈도 시각화**</font>

**범주별 고객 이탈 분석**

---
이 보고서에서는 Telco 고객 이탈 데이터셋을 분석하여 고객 이탈 행동을 이해합니다. "Churn Label"(이탈 여부)과 다양한 범주형 변수의 관계를 분석했습니다.

(이하 각 범주별 해석은 그대로 두고, 필요시 한글로 번역)

<font color=#0b5bdb>**4. 이탈률 분석**</font>

In [ ]:
# 이탈 여부 비율 확인
churn_label_count = df["Churn Label"].value_counts()
fig = plt.figure(figsize=(7, 4), dpi=100)
plt.pie(
    churn_label_count,
    autopct="%.2f%%",
    labels=churn_label_count.index,
    colors=["lightblue", "red"],
    explode=[0, 0.1],
)
plt.title("이탈 여부 파이차트")
plt.show()

위 그래프에서 이탈률은 `26.54%`임을 알 수 있습니다.

# 고객 이탈 사유 분석

<font color=#0b5bdb>**1. 데이터프레임으로 결과 표시**</font>

<font color=#0b5bdb>**2. 트리맵으로 결과 시각화**</font>

**결과 해석**

---
이탈 사유 분석을 통해 고객 이탈에 영향을 주는 요인을 파악할 수 있습니다.  
예를 들어 `"고객지원 담당자의 태도"`가 10.27%로 가장 큰 이탈 사유임을 알 수 있습니다.  
(이하 해석은 한글로 자연스럽게 번역)

# 상관관계 분석

<p style="font-size: 16px;">피어슨 상관계수로 컬럼 간 상관관계를 파악하여 PCA(주성분 분석) 적용 여부를 판단합니다.</p>

**상관계수 해석**

---
>상관계수는 데이터셋 내 다양한 속성 간의 관계를 파악하는 데 도움을 줍니다.  
여기서는 몇 가지 주목할 만한 상관관계를 설명합니다:

1. `가입 개월 수`와 `총 청구금액`(0.825): 매우 강한 양의 상관관계
2. `가입 개월 수`와 `월 청구금액`(0.248): 약한 양의 상관관계
3. `총 청구금액`과 `CLTV`(0.341): 양의 상관관계

이러한 상관관계는 고객 이탈 예측 및 가치 분석에 활용될 수 있습니다.

# 피처 엔지니어링

<font color=#0b5bdb>**1. 분석에 불필요한 컬럼 제거**</font>

<font color=#0b5bdb>**2. 결측치 처리**</font>

**위에서 보듯 `Total Charges` 컬럼에 11개, `Churn Reason` 컬럼에 5174개의 결측치가 있습니다.**

`Churn Reason` 컬럼은 대부분의 고객이 이탈하지 않아 결측치가 많습니다.

결측치 처리 방법:
- 1. 제거
- 2. 대체(평균 등)
- 3. 마스킹

**아래는 `Total Charges` 컬럼을 평균값으로 대체한 예시입니다.**

<font color=#0b5bdb>**3. 이상치 분석**</font>

박스플롯에서 `Total Charges`와 `Tenure Months` 컬럼에 이상치가 있음을 알 수 있습니다.  
또한, 허용 가능한 왜도는 `-0.5 ~ +0.5`이나, `Total Charges`는 왜도가 0.96으로 높습니다.

**해결책**: 로그 변환을 적용해 이상치와 정규성 문제를 완화합니다.

위 그래프에서 `Total Charges` 컬럼의 왜도가 0.76으로 줄어든 것을 확인할 수 있습니다.

<font color=#0b5bdb>**4. 범주형 컬럼 인코딩**</font>

원-핫 인코딩 적용 결과 컬럼 수가 1175개 늘어나 데이터의 차원이 매우 높아졌습니다.

# 가설 검정

**가설 검정**

---
**가설 1: 전화 서비스가 이탈에 영향을 미치는가?**
- 귀무가설(H0): 전화 서비스와 이탈 간에 유의미한 관계가 없다.
- 대립가설(H1): 전화 서비스와 이탈 간에 유의미한 관계가 있다.

**가설 2: 계약 유형이 이탈에 영향을 미치는가?**
- 귀무가설(H0): 계약 유형이 이탈 가능성에 영향을 주지 않는다.
- 대립가설(H1): 계약 유형이 이탈 가능성에 유의미한 영향을 준다.

**가설 3: 고령자 여부가 이탈에 영향을 미치는가?**
- 귀무가설(H0): 고령자 여부가 이탈 가능성에 영향을 주지 않는다.
- 대립가설(H1): 고령자 여부가 이탈 가능성에 유의미한 영향을 준다.

<p>카이제곱 검정: 범주형 컬럼과 이탈 여부의 관계를 검정합니다.
    
$\chi^{2} = \sum_{i=1}^{n} \frac{(O_i-E_i)^2}{E_i}$

**가설 검정 결과 해석**

---
**가설 1 (전화 서비스):**
- **결론:** p값이 0.34로 유의수준보다 크므로 귀무가설을 기각하지 못합니다. 즉, 전화 서비스와 이탈 간에 유의미한 관계가 없습니다.

**가설 2 (계약 유형):**
- **결론:** p값이 0.00으로 유의수준보다 작으므로 귀무가설을 기각합니다. 즉, 계약 유형이 이탈에 유의미한 영향을 미칩니다.

**가설 3 (고령자 여부):**
- **결론:** p값이 0.00으로 유의수준보다 작으므로 귀무가설을 기각합니다. 즉, 고령자 여부가 이탈에 유의미한 영향을 미칩니다.

# 데이터 분석 다음 단계 제안

---

- **고객 세분화**: 다양한 고객군을 분석해 맞춤 전략을 수립할 수 있습니다.
- **주성분 분석(PCA)**: 차원 축소로 중요한 특성에 집중할 수 있습니다.
- **머신러닝 모델링**: 로지스틱 회귀, 결정트리, 랜덤포레스트, 그래디언트 부스팅 등 분류 알고리즘으로 이탈 예측 모델을 구축할 수 있습니다.

# 데이터 품질 요약

---
제공된 데이터는 고객 이탈 분석에 적합한 다양한 변수와 속성을 포함하고 있어 품질이 비교적 높아 보입니다.  
다만, 결측치, 이상치, 불일치 등 데이터 정제 과정이 필요할 수 있습니다.  
분석 및 의사결정의 품질은 데이터의 정확성과 대표성, 그리고 선택한 특성의 적합성에 달려 있습니다.